In [1]:
import xarray as xr
import numpy as np
from pathlib import Path
import numpy as np
import cftime

In [2]:
out_dir = Path("/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/benchmarks/states_stabilisation/updated/")
out_dir.mkdir(exist_ok=True, parents=True)

# Ens files
ens_dir = Path("/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/pipeline/3.benchmark/ilamb/ensmean_files/S3")
ens_clitter = xr.open_dataset(ens_dir / "ENSMEAN_S3_cLitter.nc")["cLitter"]
ens_csoil = xr.open_dataset(ens_dir / "ENSMEAN_S3_cSoil.nc")["cSoil"]
ens_cveg = xr.open_dataset(ens_dir / "ENSMEAN_S3_cVeg.nc")["cVeg"]
ens_mrso = xr.open_dataset(ens_dir / "ENSMEAN_S3_mrso.nc")["mrso"]
ens_lai = xr.open_dataset(ens_dir / "ENSMEAN_S3_lai.nc")["lai"]
ens_ctotal = xr.open_dataset(ens_dir / "ENSMEAN_S3_cTotal_monthly.nc")["cTotal"]
ens_das = {"cLitter": ens_clitter,
            "cSoil": ens_csoil,
            "cVeg": ens_cveg,
            "mrso": ens_mrso,
            "lai": ens_lai,
            "cTotal": ens_ctotal}

# Stable files
stable_dir = "/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/data/predictions/carry/32_year_full_time/netcdf/S3/full"
stable_lai = xr.open_dataset(f"{stable_dir}/lai.nc")["lai"]
stable_cLitter = xr.open_dataset(f"{stable_dir}/cLitter.nc")["cLitter"]
stable_cSoil = xr.open_dataset(f"{stable_dir}/cSoil.nc")["cSoil"]
stable_cVeg = xr.open_dataset(f"{stable_dir}/cVeg.nc")["cVeg"]
stable_mrso = xr.open_dataset(f"{stable_dir}/mrso.nc")["mrso"]
stable_ctotal = xr.open_dataset(f"{stable_dir}/cTotal.nc")["cTotal"]
stables_das = {"cLitter": stable_cLitter,
                "cSoil": stable_cSoil,
                "cVeg": stable_cVeg,
                "mrso": stable_mrso,    
                "lai": stable_lai,
                "cTotal": stable_ctotal}



In [3]:
# Base files
base_dir = Path("/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/data/predictions/carry/base_model/netcdf/S3/full")
base_lai = xr.open_dataset(f"{base_dir}/lai.nc")["lai"]
base_cLitter = xr.open_dataset(f"{base_dir}/cLitter.nc")["cLitter"]
base_cSoil = xr.open_dataset(f"{base_dir}/cSoil.nc")["cSoil"]
base_cVeg = xr.open_dataset(f"{base_dir}/cVeg.nc")["cVeg"]
base_mrso = xr.open_dataset(f"{base_dir}/mrso.nc")["mrso"]
base_ctotal = xr.open_dataset(f"{base_dir}/cTotal.nc")["cTotal"]
base_das = {"cLitter": base_cLitter,
             "cSoil": base_cSoil,
             "cVeg": base_cVeg,
             "mrso": base_mrso,
             "lai": base_lai,
             "cTotal": base_ctotal}

# tvt
tvt_mask = xr.open_dataset("/Net/Groups/BGI/people/ecathain/TRENDY_Emulator_Scripts/NewModel/data/masks/tvt_mask.nc")["tvt_mask"]

In [4]:
# Precompute land mask once (boolean)
land_mask = tvt_mask.isin([0, 1, 2])

def apply_mask(dct, mask):
    """Return a new dict with mask applied to each DataArray."""
    return {var: da.where(mask) for var, da in dct.items()}

masked_ens    = apply_mask(ens_das,     land_mask)
masked_stable = apply_mask(stables_das, land_mask)
masked_base   = apply_mask(base_das,    land_mask)


Convert annual to monthly

In [5]:

for var, da in masked_base.items():
    out_path = out_dir / "base" / f"{var}.nc"
    out_path.parent.mkdir(exist_ok=True, parents=True)
    da.to_netcdf(out_path)

for var, da in masked_ens.items():
    
    out_path = out_dir / "ensmean" / f"ENSMEAN_S3_{var}.nc"
    out_path.parent.mkdir(exist_ok=True, parents=True)
    da.to_netcdf(out_path)

In [6]:
for var, da in masked_stable.items():

    out_path = out_dir / "stable" / f"{var}.nc"
    out_path.parent.mkdir(exist_ok=True, parents=True)
    da.to_netcdf(out_path)